<a href="https://colab.research.google.com/github/llindish/Complete-Python-3-Bootcamp/blob/master/WTD_model/GEE_code/FlandersMoss_S2_mosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import ee
import geemap

In [4]:
##Authentification needed only once, first time you run the colab code
from google.colab import auth
auth.authenticate_user()

In [24]:
cloud_project = 'geographic-information-systems'
ee.Initialize(project=cloud_project)
geemap.ee_initialize(project=cloud_project)

In [63]:

#monthly Senitnel-2 iamgery with CS+ mask for Flanders moss

# !Specify the asset (only for vosialisation here on GEEmap, can be skipped)
#asset (.shp file of the loggers stored on GEE as asset)
asset_id = 'users/llindish/PA_sites/PA_Flanderssmoss'

# !Specify the year.
year = 2021;

# !Specify the name of your bucket(where would you like to export) and the specific location in the bucket
bucket = 'soilmoisture-jbdsag'
path = 'GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021'

# make sure the asset is a feature collection
geometry = ee.FeatureCollection(asset_id)

# !Specify the boundary box (the images area is based on these coordinates)
coords = [[-4.235209770562007,56.14116342121212],
          [-4.160708732964351,56.14116342121212],
          [-4.160708732964351,56.176725376357545],
          [-4.235209770562007,56.176725376357545],
          [-4.235209770562007,56.14116342121212]]

# Cloud Score+ image collection.
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

# Harmonized Sentinel-2 Level 2A collection.
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Link S2 and CS+ results.
linkedCollection = s2.linkCollection(csPlus, ['cs', 'cs_cdf'])

# Masks pixels with low CS+ QA scores.
def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.60
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)

# NDVI and NDWI calculation function.
def addIndices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B8', 'B3']).rename('NDWI')
    return image.addBands(ndvi).addBands(ndwi)

# Create the polygon
aoi = ee.Geometry.Polygon(coords)

# Convert the year to ee.Date objects
start_date = ee.Date.fromYMD(year, 1, 1)
end_date = ee.Date.fromYMD(year, 12, 31)

# Apply the cloud mask and NDVI calculation.
withIndices = linkedCollection.filterBounds(aoi).filterDate(start_date, end_date).map(maskLowQA).map(addIndices)

# Function to filter images by month and compute the median
def monthly_median(year, month):
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, 'month')
    monthly_images = withNDVI.filterDate(start_date, end_date)
    median_image = monthly_images.median()
    return median_image.set('system:time_start', start_date.millis(), 'month', month)

# Create a list of months
months = ee.List.sequence(1, 12)

# Map the function over the list of months to get a new ImageCollection
monthly_medians = ee.ImageCollection.fromImages(months.map(lambda m: monthly_median(year, m)))

# Export each image in the ImageCollection
def export_image(img):
    """Inner function to export a single image."""
    month = ee.Number(img.get('month')).format('%02d').getInfo()
    # Construct the file name prefix.
    file_name_prefix = f"{path}/FM_S2_monthly_{year}_{month}"
    print(f"Exporting {file_name_prefix}")
     # Cast all bands to Float32.
    img = img.float()
    export_options = {
        'image': img,
        'bucket': bucket_name,
        'fileNamePrefix': file_name_prefix,
        'region': aoi,
        'scale': 10,
        'maxPixels': 1e10,
        'crs': 'EPSG:27700'
    }
    task = ee.batch.Export.image.toCloudStorage(**export_options)
    task.start()

# Initialize your ImageCollection
image_collection = monthly_medians
# Specify your bucket name
bucket_name = bucket
folder_path = path

# Call the function
monthly_medians_list = monthly_medians.toList(monthly_medians.size())
for i in range(monthly_medians_list.size().getInfo()):
    img = ee.Image(monthly_medians_list.get(i))
    export_image(img)

Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_01
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_02
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_03
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_04
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_05
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_06
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_07
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_08
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_09
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_10
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM_S2_monthly_2021_11
Exporting GEE_exports_LT/S2/Flanders_Moss/Brit_grid_27700/2021/FM

In [62]:
#Visualisatioz on GEEmap

map = geemap.Map()
url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
map.add_tile_layer(url, name="Google Satellite", attribution="Google")

# Centre the map view over the ROI
map.centerObject(geometry, 13)

# !Specify the month
month = 3

# Convert the year and month to ee.Date objects
start_date = ee.Date.fromYMD(year, month, 1)
end_date = start_date.advance(1, 'month')

# Filter the collection for the specified month and create a mosaic
monthly_mosaic = image_collection.filterBounds(my_aoi).filterDate(start_date, end_date).mosaic()

# Clip the mosaic to your area of interest
clipped_mosaic = monthly_mosaic.clip(my_aoi)

# Select the NDVI band
ndvi_band = clipped_mosaic.select('NDVI')

# Define visualization parameters
viz_params = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add the NDVI band to the map
map.addLayer(ndvi_band, viz_params, 'NDVI June')
map.addLayer(geometry, {'color': 'red'}, 'Loggers')

map

Map(center=[56.16272439597686, -4.19856006658251], controls=(WidgetControl(options=['position', 'transparent_b…